# 🔬 Advanced Machine Learning for COVID-19 Chest X-Ray Classification

## 📋 Comprehensive Guide: Ensemble Methods & Deep Learning Fine-Tuning

**Auteurs**: Équipe DS_COVID  
**Date**: 15 octobre 2025  
**Branche**: ReVamp

---

### 🎯 **Objectifs du notebook**

Ce notebook présente une approche complète du machine learning pour la classification de radiographies pulmonaires COVID-19, couvrant:

1. **🌳 Méthodes d'Ensemble** : Bagging, Boosting, Stacking
2. **🧠 Deep Learning** : Transfer Learning & Fine-Tuning  
3. **⚙️ Optimisation** : Hyperparamètres, Cross-validation
4. **📊 Évaluation** : Métriques avancées, Visualisations

### 📁 **Dataset**
- **Source**: COVID-19_Radiography_Dataset
- **Classes**: COVID, Normal, Lung_Opacity, Viral Pneumonia
- **Type**: Images radiographiques pulmonaires

---

### 🗂️ **Structure du notebook**

| Section | Technique | Description |
|---------|-----------|-------------|
| 1-3 | **Setup & EDA** | Chargement, exploration, preprocessing |
| 4-6 | **Ensemble Methods** | Bagging, Boosting, comparaisons |
| 7-10 | **Deep Learning** | CNN, Transfer Learning, Fine-Tuning |
| 11-13 | **Optimization & Evaluation** | Tuning, métriques, visualisations |

## 📦 Section 1: Setup and Data Loading

## 🚀 Version Google Colab

**Pour utiliser ce notebook sur Google Colab, exécutez cette cellule au lieu de la cellule de configuration normale :**

### 📋 Instructions pour Google Colab

#### ✅ Ce qui change sur Colab vs Local :

1. **📂 Gestion des données** :
   - Local : Dataset dans `data/raw/COVID-19_Radiography_Dataset/`
   - Colab : **Extraction automatique** d'`archive_covid.zip` depuis Drive

2. **📦 Installation** :
   - Local : `pip install -e .` fait une seule fois
   - Colab : Installation automatique dans la cellule

3. **🔧 Configuration** :
   - Local : Chargement automatique `.env`
   - Colab : Package ds-covid + extraction ZIP

#### 🚀 Ce que fait automatiquement la cellule Colab :

1. **📥 Clone le repo** : `git clone https://github.com/L-Poca/DS_COVID.git`
2. **📦 Extrait archive_covid.zip** : Depuis `MyDrive/archive_covid.zip`
3. **🔍 Recherche intelligente** : Trouve automatiquement le dossier COVID
4. ** Installe les dépendances** : `pip install -r requirements.txt`
5. **🔧 Installe le package** : `pip install -e .` 
6. **⚙️ Configure les chemins** : `DATA_DIR` pointe vers les données extraites
7. **✅ Vérifie les données** : Compte les images par classe

#### 💾 Votre fichier sur Drive :

```
Drive/
├── MyDrive/
│   └── archive_covid.zip  ← Votre fichier ZIP
```

**Le script fait automatiquement** :
1. **Extrait** `archive_covid.zip` vers `/content/temp_covid_extract/`
2. **Recherche** le dossier COVID (plusieurs patterns supportés)
3. **Déplace** vers `/content/DS_COVID/data/raw/COVID-19_Radiography_Dataset/`
4. **Nettoie** le dossier temporaire

#### 🔍 Recherche intelligente :

Le script cherche automatiquement :
- `COVID-19_Radiography_Dataset/`
- Dossiers contenant `*COVID*`
- Dossiers contenant `*radiography*`
- Dossiers contenant `*chest*`

#### 💡 Avantages :

- ✅ **Simple** : Juste déposer archive_covid.zip dans Drive
- ✅ **Automatique** : Extraction et organisation automatiques
- ✅ **Robuste** : Recherche intelligente de la structure
- ✅ **Rapide** : Pas besoin de créer des dossiers manuellement

### 🚀 Optimisations Colab Pro

#### ⚡ Paramètres ambitieux pour GPU puissant :

**🖼️ Images :**
- `IMG_SIZE`: `(256, 256)` ← Puissance de 2 (optimal GPU)
- `BATCH_SIZE`: `64` ← Plus gros batch (GPU T4/V100)

**🎯 Training :**
- `EPOCHS`: `100` ← Training long (temps illimité Colab Pro)
- `MAX_IMAGES`: `5000` ← Dataset complet par classe

**🤖 ML traditionnel :**
- Random Forest : `500 estimateurs` (vs 200)
- XGBoost : `300 estimateurs` (vs 100)  
- Gradient Boosting : `300 estimateurs`
- CV : `5 folds` (vs 3)

**🧠 Deep Learning :**
- Architectures : `EfficientNetB3`, `ResNet152V2`, `VGG19`, `DenseNet201`
- Mixed precision : `float16` (accélération GPU)
- Fine-tuning : `20 couches` (vs 10)
- Data augmentation : Rotation, zoom, flip, brightness

#### 💡 Pourquoi ces optimisations ?

1. **GPU T4/V100** : Plus de mémoire et compute → batch size plus gros
2. **Temps illimité** : Colab Pro permet training long → plus d'epochs
3. **Puissance de 2** : `256x256` optimise les opérations GPU vs `224x224`
4. **Mixed precision** : Accélération x1.5-2x sur GPU récents
5. **Architectures avancées** : Plus performantes qu'EfficientNetB0/ResNet50

In [ ]:
# ===================================
# CONFIGURATION POUR GOOGLE COLAB
# ===================================
# ⚠️ Exécutez cette cellule uniquement sur Google Colab
# Pour l'environnement local, utilisez la cellule suivante

import os
import sys

# Vérification si on est sur Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 Google Colab détecté")
    
    # 1. Connexion à Google Drive (pour accéder aux données)
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. Clonage du repository depuis GitHub avec la branche ReVamp
    repo_exists = os.path.exists('/content/DS_COVID')
    if not repo_exists:
        print("📥 Clonage du repository (branche ReVamp)...")
        !git clone -b ReVamp https://github.com/L-Poca/DS_COVID.git /content/DS_COVID
        print("✅ Repository cloné (branche ReVamp)")
    else:
        print("✅ Repository déjà présent")
        # S'assurer qu'on est sur la bonne branche
        os.chdir('/content/DS_COVID')
        print("🔄 Vérification de la branche...")
        !git fetch origin
        !git checkout ReVamp
        !git pull origin ReVamp
        print("✅ Branche ReVamp mise à jour")
    
    # 3. Changement vers le dossier du projet
    os.chdir('/content/DS_COVID')
    print(f"📁 Dossier courant: {os.getcwd()}")
    
    # Vérification que pyproject.toml existe (preuve qu'on est sur ReVamp)
    if os.path.exists('pyproject.toml'):
        print("✅ pyproject.toml trouvé - Branche ReVamp confirmée")
    else:
        print("⚠️ pyproject.toml manquant - Problème de branche!")
    
    # 4. Extraction de archive_covid.zip depuis Google Drive
    print("📁 Extraction de archive_covid.zip depuis Drive...")
    
    # Chemin vers votre fichier ZIP sur Drive
    zip_file_path = "/content/drive/MyDrive/archive_covid.zip"
    
    # Dossier de destination
    local_data_dir = "/content/DS_COVID/data/raw"
    local_data_path = f"{local_data_dir}/COVID-19_Radiography_Dataset"
    
    # Création du dossier data/raw
    os.makedirs(local_data_dir, exist_ok=True)
    
    if os.path.exists(zip_file_path):
        print(f"✅ Archive trouvée: {zip_file_path}")
        
        if not os.path.exists(local_data_path):
            print("📦 Extraction de l'archive COVID...")
            import zipfile
            import shutil
            import glob
            
            # Extraction dans un dossier temporaire
            temp_extract_path = "/content/temp_covid_extract"
            
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(temp_extract_path)
                print(f"✅ Archive extraite dans {temp_extract_path}")
            
            # Recherche intelligente du dossier COVID
            search_patterns = [
                f"{temp_extract_path}/**/COVID-19_Radiography_Dataset",
                f"{temp_extract_path}/**/*COVID*",
                f"{temp_extract_path}/**/*radiography*",
                f"{temp_extract_path}/**/*chest*"
            ]
            
            covid_source = None
            for pattern in search_patterns:
                possible_dirs = glob.glob(pattern, recursive=True)
                possible_dirs = [d for d in possible_dirs if os.path.isdir(d)]
                if possible_dirs:
                    covid_source = possible_dirs[0]
                    break
            
            if covid_source:
                print(f"✅ Dossier COVID trouvé: {covid_source}")
                
                # Vérification de la structure (doit contenir COVID, Normal, etc.)
                subdirs = [d.name for d in Path(covid_source).iterdir() if d.is_dir()]
                print(f"   Sous-dossiers: {subdirs}")
                
                # Déplacement vers la destination finale
                if os.path.exists(local_data_path):
                    shutil.rmtree(local_data_path)
                shutil.move(covid_source, local_data_path)
                
                # Nettoyage du dossier temporaire  
                shutil.rmtree(temp_extract_path)
                print(f"✅ Données COVID disponibles: {local_data_path}")
            else:
                print("⚠️ Structure du ZIP non reconnue automatiquement")
                # Affichage du contenu pour debug
                for root, dirs, files in os.walk(temp_extract_path):
                    if dirs:
                        print(f"   📁 {root}: {dirs[:3]}...")  # Premières 3 dossiers
                        break
                print(f"   Vérifiez manuellement dans: {temp_extract_path}")
        else:
            print(f"✅ Données déjà extraites: {local_data_path}")
    else:
        print(f"❌ Archive non trouvée: {zip_file_path}")
        print("💡 Solutions possibles:")
        print("   1. Vérifiez que 'archive_covid.zip' est dans MyDrive/")
        print("   2. Ou modifiez zip_file_path avec le bon chemin")
        print("   3. Ou uploadez archive_covid.zip dans MyDrive/")
    
    # 5. Installation des dépendances depuis requirements.txt
    print("📦 Installation des dépendances...")
    !pip install -r requirements.txt
    
    # 6. Installation du package ds-covid en mode développement
    print("📦 Installation du package ds-covid...")
    !pip install -e .
    
    # 7. Vérification de l'installation et configuration
    try:
        import ds_covid
        from ds_covid import Settings
        print(f"✅ Package ds-covid v{ds_covid.__version__} installé avec succès")
        
        # Configuration optimisée pour Colab Pro
        settings = Settings()
        
        # Adaptation des chemins pour Colab
        from pathlib import Path
        PROJECT_ROOT = Path('/content/DS_COVID')
        
        # Vérification des données extraites
        if os.path.exists(local_data_path):
            # Structure normale: COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset/
            inner_covid_path = Path(local_data_path) / 'COVID-19_Radiography_Dataset'
            if inner_covid_path.exists():
                DATA_DIR = inner_covid_path
            else:
                DATA_DIR = Path(local_data_path)
        else:
            DATA_DIR = Path(local_data_path)
        
        MODELS_DIR = PROJECT_ROOT / 'models'
        RESULTS_DIR = PROJECT_ROOT / 'results'
        
        # Variables optimisées pour Colab Pro
        BATCH_SIZE = settings.training.batch_size      # 64
        EPOCHS = settings.training.epochs              # 100
        LEARNING_RATE = settings.training.learning_rate
        IMG_SIZE = settings.training.img_size          # (256, 256)
        IMG_CHANNELS = settings.training.img_channels
        TEST_SPLIT = settings.training.test_split
        VALIDATION_SPLIT = settings.training.validation_split
        RANDOM_SEED = settings.training.random_seed
        
        # Configuration avancée
        MAX_IMAGES_PER_CLASS = settings.data.max_images_per_class
        AUGMENTATION_PARAMS = settings.data.augmentation_params
        DL_ARCHITECTURES = settings.deep_learning.architectures
        MIXED_PRECISION = settings.deep_learning.mixed_precision
        
        # Classes
        CLASSES = settings.data.class_names
        CLASS_MAPPING = settings.data.class_mapping
        NUM_CLASSES = settings.data.num_classes
        
        print(f"✅ Configuration COLAB PRO:")
        print(f"   - Données: {DATA_DIR}")
        print(f"   - Classes: {CLASSES}")
        print(f"   - Image size: {IMG_SIZE}")
        print(f"   - Batch size: {BATCH_SIZE}")
        print(f"   - Epochs: {EPOCHS}")
        
    except ImportError as e:
        print(f"⚠️ Erreur package: {e}")
        # Configuration manuelle de fallback
        from pathlib import Path
        PROJECT_ROOT = Path('/content/DS_COVID')
        DATA_DIR = Path(local_data_path)
        MODELS_DIR = PROJECT_ROOT / 'models'
        RESULTS_DIR = PROJECT_ROOT / 'results'
        
        BATCH_SIZE = 64
        EPOCHS = 100
        LEARNING_RATE = 0.001
        IMG_SIZE = (256, 256)
        IMG_CHANNELS = 3
        TEST_SPLIT = 0.2
        VALIDATION_SPLIT = 0.2
        RANDOM_SEED = 42
        
        CLASSES = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']
        CLASS_MAPPING = {'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Viral Pneumonia': 3}
        NUM_CLASSES = len(CLASSES)
    
    # Création des dossiers
    MODELS_DIR.mkdir(parents=True, exist_ok=True)
    RESULTS_DIR.mkdir(parents=True, exist_ok=True)
    
    # Vérification finale des données
    if DATA_DIR.exists():
        subdirs = [d.name for d in DATA_DIR.iterdir() if d.is_dir()]
        print(f"   - Dossiers trouvés: {subdirs}")
        
        # Comptage rapide des images
        total_images = 0
        for subdir in subdirs[:4]:  # Premières 4 classes
            subdir_path = DATA_DIR / subdir
            if subdir_path.exists():
                images = list(subdir_path.glob('*.png')) + list(subdir_path.glob('*.jpg'))
                print(f"     {subdir}: {len(images)} images")
                total_images += len(images)
        print(f"   - Total: ~{total_images} images")
    else:
        print(f"   - ⚠️ Dossier non accessible: {DATA_DIR}")
    
else:
    print("💻 Environnement local détecté - utilisez la cellule de configuration normale")
    
# Imports communs (marchent partout)
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from PIL import Image
import pickle
import joblib
from datetime import datetime
import json

warnings.filterwarnings('ignore')

# Machine Learning
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Configuration GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f"✅ GPU détecté: {len(physical_devices)} device(s)")
else:
    print("⚠️ Pas de GPU détecté, utilisation du CPU")

if IN_COLAB:
    print(f"\n📦 Versions (Colab):")
    print(f"   - Python: {sys.version.split()[0]}")
    print(f"   - Scikit-learn: {sklearn.__version__}")
    print(f"   - TensorFlow: {tf.__version__}")
    print(f"   - OpenCV: {cv2.__version__}")
    print(f"   - NumPy: {np.__version__}")
    print(f"   - Pandas: {pd.__version__}")
    print("\n✅ Configuration Colab terminée - Données extraites depuis archive_covid.zip !")

In [ ]:
# Configuration pour Google Colab avec archive_covid.zip
import os
import sys
from google.colab import drive

print("? CONFIGURATION GOOGLE COLAB PRO")

# Monter Google Drive
drive.mount('/content/drive')

# Cloner le repo si pas déjà fait
if not os.path.exists('/content/DS_COVID'):
    print("📥 Clonage du repository...")
    !git clone https://github.com/L-Poca/DS_COVID.git /content/DS_COVID

# Aller dans le dossier du projet
os.chdir('/content/DS_COVID')

# Extraction intelligente de archive_covid.zip depuis Drive
import zipfile
import shutil

def find_and_extract_covid_archive():
    """Trouve et extrait archive_covid.zip depuis Google Drive"""
    possible_paths = [
        '/content/drive/MyDrive/archive_covid.zip',
        '/content/drive/My Drive/archive_covid.zip',
        '/content/drive/MyDrive/COVID/archive_covid.zip',
        '/content/drive/My Drive/COVID/archive_covid.zip'
    ]
    
    for zip_path in possible_paths:
        if os.path.exists(zip_path):
            print(f"📦 Archive trouvée: {zip_path}")
            
            # Créer le dossier data/raw s'il n'existe pas
            os.makedirs('/content/DS_COVID/data/raw', exist_ok=True)
            
            # Extraire l'archive
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall('/content/DS_COVID/data/raw/')
            
            # Chercher le dossier COVID-19_Radiography_Dataset
            for root, dirs, files in os.walk('/content/DS_COVID/data/raw/'):
                for dir_name in dirs:
                    if 'COVID' in dir_name and 'Radiography' in dir_name:
                        dataset_path = os.path.join(root, dir_name)
                        print(f"✅ Dataset trouvé: {dataset_path}")
                        return dataset_path
            
            print("⚠️ Dossier COVID-19_Radiography_Dataset non trouvé dans l'archive")
            return None
    
    print("❌ Archive archive_covid.zip non trouvée dans Drive")
    print("📂 Vérifiez que le fichier est dans MyDrive/")
    return None

# Extraire les données
dataset_path = find_and_extract_covid_archive()

# Installation du package
print("📦 Installation du package ds-covid...")
!pip install -e .

In [ ]:
# ===================================
# 1.1 CONFIGURATION LOCALE (VS Code / Environnement local)
# ===================================
# ⚠️ Cette cellule est pour l'environnement LOCAL avec le package ds-covid installé
# Pour Google Colab, utilisez la cellule précédente

# Configuration du package ds-covid (utilise automatiquement .env)
from ds_covid import Settings, configure_package, __version__
import ds_covid

# Imports de base
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import cv2
from PIL import Image
import pickle
import joblib
from datetime import datetime
import json
import sys

# Configuration des warnings
warnings.filterwarnings('ignore')

# Machine Learning - Scikit-learn
import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve

# Deep Learning - TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Configuration GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print(f"✅ GPU détecté: {len(physical_devices)} device(s)")
else:
    print("⚠️ Pas de GPU détecté, utilisation du CPU")

# Configuration automatique depuis .env
print(f"📦 Package ds-covid v{__version__}")
print("🔧 Chargement de la configuration depuis .env...")

# Le package charge automatiquement les settings depuis .env
settings = Settings()

# Affichage de la configuration chargée
print(f"✅ Configuration chargée depuis .env:")
print(f"   - PROJECT_ROOT: {settings.project_root}")
print(f"   - DATA_DIR: {settings.data_dir}")
print(f"   - MODELS_DIR: {settings.models_dir}")
print(f"   - RESULTS_DIR: {settings.results_dir}")

# Variables globales depuis les settings (.env)
RANDOM_SEED = settings.training.random_seed
BATCH_SIZE = settings.training.batch_size
EPOCHS = settings.training.epochs
LEARNING_RATE = settings.training.learning_rate
IMG_SIZE = settings.training.img_size
IMG_CHANNELS = settings.training.img_channels
TEST_SPLIT = settings.training.test_split
VALIDATION_SPLIT = settings.training.validation_split

# Variables de données depuis .env
MAX_IMAGES_PER_CLASS = settings.data.max_images_per_class

# Chemins depuis .env
PROJECT_ROOT = Path(settings.project_root)
DATA_DIR = Path(settings.data_dir)
MODELS_DIR = Path(settings.models_dir)
RESULTS_DIR = Path(settings.results_dir)

# Vérification que les chemins existent
print(f"\n📁 Vérification des chemins (.env):")
for name, path in [('PROJECT_ROOT', PROJECT_ROOT), ('DATA_DIR', DATA_DIR), 
                   ('MODELS_DIR', MODELS_DIR), ('RESULTS_DIR', RESULTS_DIR)]:
    if path.exists():
        print(f"   ✅ {name}: {path}")
    else:
        print(f"   ❌ {name}: {path} (sera créé si nécessaire)")
        if name in ['MODELS_DIR', 'RESULTS_DIR']:
            path.mkdir(parents=True, exist_ok=True)
            print(f"   ✅ {name}: Créé avec succès")

# Classes du dataset COVID (depuis la configuration)
CLASSES = settings.data.class_names
CLASS_MAPPING = settings.data.class_mapping
NUM_CLASSES = settings.data.num_classes

# Résumé de la configuration
print(f"\n🎯 Configuration finale (depuis .env):")
print(f"   - Classes ({NUM_CLASSES}): {CLASSES}")
print(f"   - Image size: {IMG_SIZE}")
print(f"   - Batch size: {BATCH_SIZE}")
print(f"   - Epochs: {EPOCHS}")
print(f"   - Max images/classe: {MAX_IMAGES_PER_CLASS}")

# Pour Colab Pro - Configuration optimisée
print("\n🚀 OPTIMISATIONS COLAB PRO ACTIVÉES")
print("✅ Mixed precision activée (float16)")

print("\n🎯 PARAMÈTRES COLAB PRO:")
print(f"   - Image size: {IMG_SIZE} ← Puissance de 2 (optimal GPU)")
print(f"   - Batch size: {BATCH_SIZE} ← Plus gros (GPU puissant)")
print(f"   - Epochs: {EPOCHS} ← Plus long (temps illimité)")
print(f"   - Max images: {MAX_IMAGES_PER_CLASS} ← Dataset complet")

# Test mémoire GPU
if tf.config.list_physical_devices('GPU'):
    # Activer mixed precision si GPU disponible
    from tensorflow.keras import mixed_precision
    mixed_precision.set_global_policy('mixed_float16')
    print("✅ Mixed precision float16 activée")

print("🎉 Configuration terminée!")

⚠️ Pas de GPU détecté, utilisation du CPU
📦 Package ds-covid v0.1.0
🔧 Chargement de la configuration depuis .env...
📄 Fichier .env chargé: /home/cepa/DST/projet_DS/DS_COVID/.env
✅ Configuration chargée depuis .env:
   - PROJECT_ROOT: /home/cepa/DST/projet_DS/DS_COVID
   - DATA_DIR: /home/cepa/DST/projet_DS/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset
   - MODELS_DIR: /home/cepa/DST/projet_DS/DS_COVID/models
   - RESULTS_DIR: /home/cepa/DST/projet_DS/DS_COVID/reports

📁 Vérification des chemins (.env):
   ✅ PROJECT_ROOT: /home/cepa/DST/projet_DS/DS_COVID
   ✅ DATA_DIR: /home/cepa/DST/projet_DS/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset
   ✅ MODELS_DIR: /home/cepa/DST/projet_DS/DS_COVID/models
   ✅ RESULTS_DIR: /home/cepa/DST/projet_DS/DS_COVID/reports

🎯 Configuration finale (depuis .env):
   - Classes (4): ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']
   - Mapping: {'COVID': 0, 'Lung_Opacity': 1, 'Normal': 2, 'Vir

In [7]:
Path.cwd()

PosixPath('/home/cepa/DST/projet_DS/DS_COVID/notebooks')

In [6]:
# ===================================
# 1.2 CONFIGURATION CENTRALISÉE AVEC .ENV
# ===================================

# Import du gestionnaire de configuration (chemin relatif)
# Détection automatique du chemin src depuis le notebook
notebook_dir = Path.cwd() if 'notebooks' in str(Path.cwd()) else Path(__file__).parent
project_root = notebook_dir.parent if notebook_dir.name == 'notebooks' else notebook_dir
src_path = project_root / 'src'

sys.path.append(str(src_path))
from config import config_manager, get_config, setup_environment

print("🔧 Chargement de la configuration centralisée...")

# Affichage du résumé de configuration
config_manager.print_summary()

# Création des répertoires nécessaires
config_manager.create_directories()

# Configuration de l'environnement
setup_environment()

# Récupération des variables de configuration
PROJECT_ROOT = get_config('paths', 'project_root')
DATA_DIR = get_config('paths', 'data_dir')
MODELS_DIR = get_config('paths', 'models_dir')
RESULTS_DIR = get_config('paths', 'results_dir')

# Configuration d'images
IMG_SIZE = get_config('image', 'img_size')
IMG_CHANNELS = get_config('image', 'img_channels')

# Paramètres d'entraînement depuis .env
BATCH_SIZE = get_config('training', 'batch_size')
EPOCHS = get_config('training', 'epochs')
LEARNING_RATE = get_config('training', 'learning_rate')
VALIDATION_SPLIT = get_config('training', 'validation_split')
TEST_SPLIT = get_config('training', 'test_split')
RANDOM_SEED = get_config('training', 'random_seed')

# Classes depuis la configuration
CLASSES = get_config('classes', 'class_names')
CLASS_MAPPING = get_config('classes', 'class_mapping')

print("\n✅ Configuration chargée depuis .env!")
print(f"📊 Paramètres principaux:")
print(f"   🖼️ Taille d'image: {IMG_SIZE}")
print(f"   📦 Batch size: {BATCH_SIZE}")
print(f"   🔄 Époques: {EPOCHS}")
print(f"   🎯 Classes: {len(CLASSES)} ({', '.join(CLASSES)})")
print(f"   📁 Répertoire de données: {DATA_DIR}")

# Configuration des seeds pour reproductibilité
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

print(f"🎲 Seed configuré: {RANDOM_SEED}")
print("🔧 Environnement prêt avec configuration centralisée!")

✅ Variables d'environnement chargées depuis: /home/cepa/DST/projet_DS/DS_COVID/.env
🔧 Chargement de la configuration centralisée...
🔧 CONFIGURATION DU PROJET DS_COVID

📁 CHEMINS:
   ✅ project_root: /home/cepa/DST/projet_DS/DS_COVID
   ✅ data_dir: /home/cepa/DST/projet_DS/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset
   ✅ models_dir: /home/cepa/DST/projet_DS/DS_COVID/models
   ✅ results_dir: /home/cepa/DST/projet_DS/DS_COVID/reports
   ✅ notebooks_dir: /home/cepa/DST/projet_DS/DS_COVID/notebooks

🖼️ IMAGES:
   📐 Taille: (224, 224)
   🎨 Canaux: 3

🎯 ENTRAÎNEMENT:
   📊 Batch size: 32
   🔄 Époques: 50
   📈 Learning rate: 0.001

🏷️ CLASSES:
   📋 4 classes: COVID, Lung_Opacity, Normal, Viral Pneumonia
📁 data_dir: /home/cepa/DST/projet_DS/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset
📁 models_dir: /home/cepa/DST/projet_DS/DS_COVID/models
📁 results_dir: /home/cepa/DST/projet_DS/DS_COVID/reports
📁 notebooks_dir: /home/cepa/DST/projet_D

In [ ]:
# ===================================
# 1.2 CONFIGURATION DES CHEMINS ET DATASET
# ===================================

# Chemins du projet
PROJECT_ROOT = Path("/home/cepa/DST/projet_DS/DS_COVID")
DATA_DIR = PROJECT_ROOT / "data" / "raw" / "COVID-19_Radiography_Dataset" / "COVID-19_Radiography_Dataset"
MODELS_DIR = PROJECT_ROOT / "models"
RESULTS_DIR = PROJECT_ROOT / "reports"

# Création des dossiers de sortie
MODELS_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

print(f"📁 Répertoire du projet: {PROJECT_ROOT}")
print(f"📁 Répertoire des données: {DATA_DIR}")
print(f"📁 Répertoire des modèles: {MODELS_DIR}")

# Définition des classes
CLASSES = ["COVID", "Lung_Opacity", "Normal", "Viral Pneumonia"]
CLASS_MAPPING = {cls: idx for idx, cls in enumerate(CLASSES)}

print(f"🏷️ Classes détectées: {CLASSES}")
print(f"🔢 Mapping des classes: {CLASS_MAPPING}")

# Paramètres globaux
IMG_SIZE = (224, 224)  # Taille standard pour les modèles pré-entraînés
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.2

print(f"⚙️ Taille d'image: {IMG_SIZE}")
print(f"⚙️ Batch size: {BATCH_SIZE}")
print(f"⚙️ Nombre d'époques: {EPOCHS}")

# Vérification de l'existence des données
if DATA_DIR.exists():
    print("✅ Répertoire de données trouvé")
    for class_name in CLASSES:
        class_path = DATA_DIR / class_name / "images"
        if class_path.exists():
            n_images = len(list(class_path.glob("*.png")))
            print(f"   📊 {class_name}: {n_images} images")
        else:
            print(f"   ❌ {class_name}: répertoire non trouvé")
else:
    print("❌ Répertoire de données non trouvé!")
    print(f"   Vérifiez le chemin: {DATA_DIR}")

In [ ]:
# ===================================
# 1.3 FONCTIONS UTILITAIRES POUR LE CHARGEMENT
# ===================================

def load_image_paths_and_labels(data_dir, classes):
    """
    Charge les chemins des images et leurs labels
    
    Returns:
        tuple: (image_paths, labels, class_counts)
    """
    image_paths = []
    labels = []
    class_counts = {}
    
    for class_name in classes:
        class_dir = data_dir / class_name / "images"
        if not class_dir.exists():
            continue
            
        # Récupération des images
        image_files = list(class_dir.glob("*.png"))
        class_counts[class_name] = len(image_files)
        
        # Ajout des chemins et labels
        for img_path in image_files:
            image_paths.append(str(img_path))
            labels.append(class_name)
    
    return image_paths, labels, class_counts

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    """
    Charge et préprocesse une image
    
    Args:
        image_path (str): Chemin vers l'image
        target_size (tuple): Taille cible (largeur, hauteur)
    
    Returns:
        np.array: Image préprocessée
    """
    try:
        # Chargement avec PIL
        img = Image.open(image_path).convert('RGB')
        
        # Redimensionnement
        img = img.resize(target_size)
        
        # Conversion en array numpy
        img_array = np.array(img)
        
        # Normalisation [0, 1]
        img_array = img_array.astype(np.float32) / 255.0
        
        return img_array
    
    except Exception as e:
        print(f"Erreur lors du chargement de {image_path}: {e}")
        return None

def create_balanced_subset(image_paths, labels, max_per_class=500):
    """
    Crée un sous-ensemble équilibré du dataset
    
    Args:
        image_paths (list): Liste des chemins d'images
        labels (list): Liste des labels
        max_per_class (int): Nombre maximum d'images par classe
    
    Returns:
        tuple: (subset_paths, subset_labels)
    """
    df = pd.DataFrame({'path': image_paths, 'label': labels})
    
    # Échantillonnage équilibré
    balanced_df = df.groupby('label').apply(
        lambda x: x.sample(n=min(len(x), max_per_class), random_state=42)
    ).reset_index(drop=True)
    
    return balanced_df['path'].tolist(), balanced_df['label'].tolist()

print("🔧 Fonctions utilitaires définies")

## 📊 Section 2: Exploratory Data Analysis

In [ ]:
# ===================================
# 2.1 CHARGEMENT ET ANALYSE DE LA DISTRIBUTION
# ===================================

# Chargement des chemins et labels
print("📂 Chargement des données...")
image_paths, labels, class_counts = load_image_paths_and_labels(DATA_DIR, CLASSES)

print(f"📊 Total d'images: {len(image_paths)}")
print(f"📊 Total de labels: {len(labels)}")

# Création du DataFrame pour l'analyse
df_analysis = pd.DataFrame({
    'image_path': image_paths,
    'label': labels
})

# Analyse de la distribution des classes
print("\n🏷️ Distribution des classes:")
class_distribution = df_analysis['label'].value_counts()
print(class_distribution)

# Visualisation de la distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Graphique en barres
class_distribution.plot(kind='bar', ax=ax1, color='skyblue', alpha=0.8)
ax1.set_title('Distribution des Classes', fontsize=14, fontweight='bold')
ax1.set_xlabel('Classes', fontsize=12)
ax1.set_ylabel('Nombre d\'images', fontsize=12)
ax1.tick_params(axis='x', rotation=45)

# Graphique en secteurs
ax2.pie(class_distribution.values, labels=class_distribution.index, autopct='%1.1f%%', 
        startangle=90, colors=plt.cm.Set3.colors)
ax2.set_title('Répartition des Classes (%)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Statistiques détaillées
print("\n📈 Statistiques détaillées:")
total_images = len(image_paths)
for class_name, count in class_distribution.items():
    percentage = (count / total_images) * 100
    print(f"   {class_name}: {count} images ({percentage:.2f}%)")

# Détection du déséquilibre
max_count = class_distribution.max()
min_count = class_distribution.min()
imbalance_ratio = max_count / min_count

print(f"\n⚖️ Ratio de déséquilibre: {imbalance_ratio:.2f}")
if imbalance_ratio > 2:
    print("⚠️  Dataset déséquilibré détecté - techniques de rééquilibrage recommandées")
else:
    print("✅ Dataset relativement équilibré")

In [ ]:
# ===================================
# 2.2 VISUALISATION DES IMAGES REPRÉSENTATIVES
# ===================================

def visualize_sample_images(image_paths, labels, classes, n_samples=3):
    """
    Visualise des échantillons d'images pour chaque classe
    """
    fig, axes = plt.subplots(len(classes), n_samples, figsize=(15, 12))
    fig.suptitle('Échantillons d\'Images par Classe', fontsize=16, fontweight='bold')
    
    df_viz = pd.DataFrame({'path': image_paths, 'label': labels})
    
    for i, class_name in enumerate(classes):
        class_images = df_viz[df_viz['label'] == class_name]['path'].tolist()
        
        # Sélection aléatoire d'échantillons
        samples = np.random.choice(class_images, min(n_samples, len(class_images)), replace=False)
        
        for j, img_path in enumerate(samples):
            try:
                # Chargement de l'image
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                # Affichage
                axes[i, j].imshow(img, cmap='gray' if len(img.shape) == 2 else None)
                axes[i, j].set_title(f'{class_name}', fontsize=10)
                axes[i, j].axis('off')
                
            except Exception as e:
                axes[i, j].text(0.5, 0.5, f'Erreur:\n{str(e)}', 
                              ha='center', va='center', transform=axes[i, j].transAxes)
                axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualisation des échantillons
print("🖼️ Visualisation des échantillons d'images...")
visualize_sample_images(image_paths, labels, CLASSES, n_samples=4)

In [ ]:
# ===================================
# 2.3 ANALYSE DES PROPRIÉTÉS DES IMAGES
# ===================================

def analyze_image_properties(image_paths, sample_size=100):
    """
    Analyse les propriétés des images (taille, intensité, etc.)
    """
    print(f"🔍 Analyse des propriétés sur un échantillon de {sample_size} images...")
    
    # Échantillonnage aléatoire
    sample_paths = np.random.choice(image_paths, min(sample_size, len(image_paths)), replace=False)
    
    properties = {
        'widths': [],
        'heights': [],
        'channels': [],
        'mean_intensities': [],
        'std_intensities': [],
        'file_sizes': []
    }
    
    for img_path in sample_paths:
        try:
            # Chargement avec PIL pour les propriétés de base
            img_pil = Image.open(img_path)
            width, height = img_pil.size
            
            # Chargement avec OpenCV pour l'analyse d'intensité
            img_cv = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            # Propriétés
            properties['widths'].append(width)
            properties['heights'].append(height)
            properties['channels'].append(len(img_pil.getbands()))
            properties['mean_intensities'].append(np.mean(img_cv))
            properties['std_intensities'].append(np.std(img_cv))
            properties['file_sizes'].append(os.path.getsize(img_path) / 1024)  # KB
            
        except Exception as e:
            print(f"Erreur avec {img_path}: {e}")
            continue
    
    return properties

# Analyse des propriétés
properties = analyze_image_properties(image_paths, sample_size=200)

# Création du DataFrame d'analyse
df_props = pd.DataFrame(properties)

# Affichage des statistiques
print("\n📊 Statistiques des propriétés d'images:")
print(df_props.describe())

# Visualisation des distributions
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Distribution des Propriétés d\'Images', fontsize=16, fontweight='bold')

# Tailles
axes[0, 0].hist(df_props['widths'], bins=20, alpha=0.7, color='skyblue')
axes[0, 0].set_title('Distribution des Largeurs')
axes[0, 0].set_xlabel('Largeur (pixels)')

axes[0, 1].hist(df_props['heights'], bins=20, alpha=0.7, color='lightgreen')
axes[0, 1].set_title('Distribution des Hauteurs')
axes[0, 1].set_xlabel('Hauteur (pixels)')

# Intensités
axes[0, 2].hist(df_props['mean_intensities'], bins=20, alpha=0.7, color='coral')
axes[0, 2].set_title('Distribution des Intensités Moyennes')
axes[0, 2].set_xlabel('Intensité Moyenne')

axes[1, 0].hist(df_props['std_intensities'], bins=20, alpha=0.7, color='gold')
axes[1, 0].set_title('Distribution des Écarts-Types d\'Intensité')
axes[1, 0].set_xlabel('Écart-Type Intensité')

# Tailles de fichiers
axes[1, 1].hist(df_props['file_sizes'], bins=20, alpha=0.7, color='mediumpurple')
axes[1, 1].set_title('Distribution des Tailles de Fichiers')
axes[1, 1].set_xlabel('Taille (KB)')

# Corrélation largeur/hauteur
axes[1, 2].scatter(df_props['widths'], df_props['heights'], alpha=0.6, color='darkblue')
axes[1, 2].set_title('Corrélation Largeur/Hauteur')
axes[1, 2].set_xlabel('Largeur (pixels)')
axes[1, 2].set_ylabel('Hauteur (pixels)')

plt.tight_layout()
plt.show()

# Détection des formats non standards
print(f"\n📐 Formats d'images détectés:")
unique_dimensions = df_props.groupby(['widths', 'heights']).size().reset_index(name='count')
print(unique_dimensions.sort_values('count', ascending=False))

## 🔧 Section 3: Data Preprocessing and Augmentation

In [ ]:
# ===================================
# 3.1 PRÉPARATION DES DONNÉES D'ENTRAÎNEMENT
# ===================================

print("🔄 Préparation du dataset pour l'entraînement...")

# Création d'un sous-ensemble équilibré pour des temps de traitement raisonnables
print("⚖️ Création d'un sous-ensemble équilibré...")
balanced_paths, balanced_labels = create_balanced_subset(
    image_paths, labels, max_per_class=1000
)

print(f"📊 Dataset équilibré: {len(balanced_paths)} images")

# Vérification de l'équilibrage
balanced_distribution = pd.Series(balanced_labels).value_counts()
print("🏷️ Nouvelle distribution:")
print(balanced_distribution)

# Encodage des labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(balanced_labels)

print(f"🔢 Classes encodées: {dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))}")

# Division train/validation/test
print("✂️ Division du dataset...")

# Première division: train+val / test
X_temp, X_test, y_temp, y_test = train_test_split(
    balanced_paths, encoded_labels,
    test_size=TEST_SPLIT,
    stratify=encoded_labels,
    random_state=42
)

# Seconde division: train / val
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=VALIDATION_SPLIT/(1-TEST_SPLIT),  # Ajustement pour avoir 20% du total
    stratify=y_temp,
    random_state=42
)

print(f"📊 Train: {len(X_train)} images")
print(f"📊 Validation: {len(X_val)} images")
print(f"📊 Test: {len(X_test)} images")

# Vérification de la stratification
print("\n🎯 Vérification de la stratification:")
for split_name, y_split in [("Train", y_train), ("Val", y_val), ("Test", y_test)]:
    distribution = pd.Series(y_split).value_counts().sort_index()
    percentages = (distribution / len(y_split) * 100).round(1)
    print(f"{split_name}: {dict(zip(label_encoder.classes_, percentages.values))}")

In [ ]:
# ===================================
# 3.2 PIPELINE DE PRÉPROCESSING D'IMAGES
# ===================================

class ImagePreprocessor:
    """Pipeline de préprocessing d'images optimisé"""
    
    def __init__(self, target_size=(224, 224), normalize=True):
        self.target_size = target_size
        self.normalize = normalize
    
    def preprocess_single_image(self, image_path):
        """Préprocesse une seule image"""
        try:
            # Chargement
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Redimensionnement
            img = cv2.resize(img, self.target_size)
            
            # Normalisation
            if self.normalize:
                img = img.astype(np.float32) / 255.0
            
            return img
        
        except Exception as e:
            print(f"Erreur preprocessing {image_path}: {e}")
            return np.zeros((*self.target_size, 3), dtype=np.float32)
    
    def preprocess_batch(self, image_paths, batch_size=32):
        """Préprocesse un lot d'images avec gestion mémoire"""
        n_images = len(image_paths)
        n_batches = (n_images + batch_size - 1) // batch_size
        
        # Initialisation du tableau de sortie
        images = np.zeros((n_images, *self.target_size, 3), dtype=np.float32)
        
        print(f"🔄 Preprocessing {n_images} images en {n_batches} batches...")
        
        for i in range(n_batches):
            start_idx = i * batch_size
            end_idx = min((i + 1) * batch_size, n_images)
            
            # Traitement du batch
            for j, img_path in enumerate(image_paths[start_idx:end_idx]):
                images[start_idx + j] = self.preprocess_single_image(img_path)
            
            if (i + 1) % 10 == 0:
                print(f"   📊 Batch {i+1}/{n_batches} terminé")
        
        return images

# Initialisation du preprocessor
preprocessor = ImagePreprocessor(target_size=IMG_SIZE, normalize=True)

# Preprocessing des données
print("🔄 Preprocessing des images...")

X_train_processed = preprocessor.preprocess_batch(X_train)
X_val_processed = preprocessor.preprocess_batch(X_val)
X_test_processed = preprocessor.preprocess_batch(X_test)

print(f"✅ Preprocessing terminé!")
print(f"📊 Shape train: {X_train_processed.shape}")
print(f"📊 Shape validation: {X_val_processed.shape}")
print(f"📊 Shape test: {X_test_processed.shape}")

# Conversion des labels en format catégoriel pour le deep learning
y_train_categorical = to_categorical(y_train, num_classes=len(CLASSES))
y_val_categorical = to_categorical(y_val, num_classes=len(CLASSES))
y_test_categorical = to_categorical(y_test, num_classes=len(CLASSES))

print(f"📊 Shape labels train: {y_train_categorical.shape}")

# Visualisation d'un échantillon preprocessé
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('Images Preprocessées - Échantillons', fontsize=14, fontweight='bold')

for i in range(8):
    row = i // 4
    col = i % 4
    
    # Sélection d'une image aléatoire
    idx = np.random.randint(0, len(X_train_processed))
    img = X_train_processed[idx]
    label = label_encoder.inverse_transform([y_train[idx]])[0]
    
    axes[row, col].imshow(img)
    axes[row, col].set_title(f'{label}', fontsize=10)
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===================================
# 3.3 DATA AUGMENTATION POUR LE DEEP LEARNING
# ===================================

# Configuration de l'augmentation de données
train_datagen = ImageDataGenerator(
    rotation_range=10,           # Rotation aléatoire jusqu'à 10°
    width_shift_range=0.1,       # Décalage horizontal 10%
    height_shift_range=0.1,      # Décalage vertical 10%
    zoom_range=0.1,              # Zoom aléatoire 10%
    horizontal_flip=True,        # Miroir horizontal
    brightness_range=[0.8, 1.2], # Variation de luminosité
    fill_mode='nearest'          # Mode de remplissage
)

# Générateur pour validation (pas d'augmentation)
val_datagen = ImageDataGenerator()

print("🔄 Configuration de l'augmentation de données")

# Démonstration de l'augmentation
def demonstrate_augmentation(X_sample, y_sample, n_augmentations=6):
    """Montre l'effet de l'augmentation sur quelques images"""
    
    fig, axes = plt.subplots(2, n_augmentations + 1, figsize=(20, 8))
    fig.suptitle('Démonstration de l\'Augmentation de Données', fontsize=16, fontweight='bold')
    
    for row in range(2):
        # Image originale
        idx = np.random.randint(0, len(X_sample))
        original_img = X_sample[idx]
        label = label_encoder.inverse_transform([y_sample[idx]])[0]
        
        axes[row, 0].imshow(original_img)
        axes[row, 0].set_title(f'Original\n{label}')
        axes[row, 0].axis('off')
        
        # Images augmentées
        img_batch = np.expand_dims(original_img, 0)
        augmented_generator = train_datagen.flow(img_batch, batch_size=1)
        
        for col in range(1, n_augmentations + 1):
            augmented_batch = next(augmented_generator)
            augmented_img = augmented_batch[0]
            
            # Clip pour éviter les valeurs hors [0,1]
            augmented_img = np.clip(augmented_img, 0, 1)
            
            axes[row, col].imshow(augmented_img)
            axes[row, col].set_title(f'Augmentée {col}')
            axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

# Démonstration
print("🖼️ Démonstration de l'augmentation...")
demonstrate_augmentation(X_train_processed, y_train, n_augmentations=5)

print("✅ Pipeline d'augmentation configuré")

## 🎯 Section 4: Baseline Models Implementation

In [ ]:
# ===================================
# 4.1 EXTRACTION DE FEATURES POUR ML TRADITIONNEL
# ===================================

def extract_traditional_features(images):
    """
    Extrait des features traditionnelles pour les modèles ML classiques
    
    Features extraites:
    - Statistiques d'intensité (moyenne, std, min, max)
    - Histogramme des niveaux de gris
    - Features de texture (Local Binary Pattern simulé)
    """
    print(f"🔍 Extraction de features pour {len(images)} images...")
    
    features_list = []
    
    for i, img in enumerate(images):
        # Conversion en niveaux de gris
        if len(img.shape) == 3:
            gray_img = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
        else:
            gray_img = (img * 255).astype(np.uint8)
        
        features = []
        
        # 1. Statistiques d'intensité
        features.extend([
            np.mean(gray_img),
            np.std(gray_img),
            np.min(gray_img),
            np.max(gray_img),
            np.percentile(gray_img, 25),
            np.percentile(gray_img, 50),
            np.percentile(gray_img, 75)
        ])
        
        # 2. Histogramme (16 bins)
        hist, _ = np.histogram(gray_img, bins=16, range=(0, 256))
        hist = hist / np.sum(hist)  # Normalisation
        features.extend(hist)
        
        # 3. Features de texture simples
        # Gradient
        grad_x = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=3)
        gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
        
        features.extend([
            np.mean(gradient_magnitude),
            np.std(gradient_magnitude)
        ])
        
        # 4. Features géométriques (moments)
        moments = cv2.moments(gray_img)
        if moments['m00'] != 0:
            cx = moments['m10'] / moments['m00']
            cy = moments['m01'] / moments['m00']
            features.extend([cx, cy])
        else:
            features.extend([0, 0])
        
        # 5. Énergie et entropie
        # Normalisation de l'image pour calculer l'entropie
        normalized = gray_img / 255.0
        entropy = -np.sum(normalized * np.log(normalized + 1e-10))
        energy = np.sum(normalized ** 2)
        
        features.extend([entropy, energy])
        
        features_list.append(features)
        
        if (i + 1) % 100 == 0:
            print(f"   📊 {i+1}/{len(images)} images traitées")
    
    feature_matrix = np.array(features_list)
    print(f"✅ Features extraites: shape {feature_matrix.shape}")
    
    return feature_matrix

# Extraction des features
print("🔍 Extraction des features traditionnelles...")
X_train_features = extract_traditional_features(X_train_processed)
X_val_features = extract_traditional_features(X_val_processed)
X_test_features = extract_traditional_features(X_test_processed)

# Normalisation des features
scaler = StandardScaler()
X_train_features_scaled = scaler.fit_transform(X_train_features)
X_val_features_scaled = scaler.transform(X_val_features)
X_test_features_scaled = scaler.transform(X_test_features)

print(f"📊 Features shape train: {X_train_features_scaled.shape}")
print(f"📊 Features normalisées avec StandardScaler")

In [ ]:
# ===================================
# 4.2 MODÈLES BASELINE TRADITIONNELS
# ===================================

# Dictionnaire pour stocker les résultats
baseline_results = {}

def evaluate_model(model, X_train, X_val, y_train, y_val, model_name):
    """Évalue un modèle et stocke les résultats"""
    
    print(f"\n🔄 Entraînement de {model_name}...")
    
    # Entraînement
    start_time = datetime.now()
    model.fit(X_train, y_train)
    training_time = datetime.now() - start_time
    
    # Prédictions
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # Métriques
    train_accuracy = accuracy_score(y_train, y_train_pred)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    
    # Scores détaillés pour la validation
    precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_val_pred, average='weighted')
    
    # Stockage des résultats
    results = {
        'model': model,
        'train_accuracy': train_accuracy,
        'val_accuracy': val_accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'training_time': training_time.total_seconds(),
        'y_val_pred': y_val_pred
    }
    
    baseline_results[model_name] = results
    
    print(f"✅ {model_name}:")
    print(f"   📊 Train Accuracy: {train_accuracy:.4f}")
    print(f"   📊 Val Accuracy: {val_accuracy:.4f}")
    print(f"   📊 F1-Score: {f1:.4f}")
    print(f"   ⏱️ Training Time: {training_time.total_seconds():.2f}s")
    
    return results

# Définition des modèles baseline
baseline_models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    ),
    'SVM': SVC(
        kernel='rbf',
        C=1.0,
        random_state=42,
        probability=True  # Pour les probabilités
    ),
    'Logistic Regression': LogisticRegression(
        max_iter=1000,
        random_state=42,
        n_jobs=-1
    ),
    'Decision Tree': DecisionTreeClassifier(
        max_depth=15,
        random_state=42
    )
}

# Entraînement et évaluation de tous les modèles
print("🚀 Entraînement des modèles baseline...")

for model_name, model in baseline_models.items():
    evaluate_model(
        model, 
        X_train_features_scaled, 
        X_val_features_scaled, 
        y_train, 
        y_val, 
        model_name
    )

print("\n✅ Tous les modèles baseline entraînés!")

## 🌳 Section 5: Bagging Ensemble Methods

In [ ]:
# ===================================
# 5.1 IMPLÉMENTATION DES MÉTHODES DE BAGGING
# ===================================

print("🌳 Implémentation des méthodes de Bagging...")

# Dictionnaire pour les résultats de bagging
bagging_results = {}

# 5.1.1 Random Forest optimisé
print("\n🔄 Random Forest optimisé...")
rf_optimized = RandomForestClassifier(
    n_estimators=200,           # Plus d'arbres
    max_depth=15,               # Profondeur contrôlée
    min_samples_split=5,        # Contrôle overfitting
    min_samples_leaf=2,         # Contrôle overfitting
    max_features='sqrt',        # Features aléatoires
    bootstrap=True,             # Bagging activé
    random_state=42,
    n_jobs=-1
)

evaluate_model(
    rf_optimized, 
    X_train_features_scaled, 
    X_val_features_scaled, 
    y_train, 
    y_val, 
    'Random Forest Optimized'
)

# 5.1.2 Extra Trees (Extremely Randomized Trees)
print("\n🔄 Extra Trees...")
extra_trees = ExtraTreesClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    bootstrap=True,             # Bagging
    random_state=42,
    n_jobs=-1
)

evaluate_model(
    extra_trees, 
    X_train_features_scaled, 
    X_val_features_scaled, 
    y_train, 
    y_val, 
    'Extra Trees'
)

# 5.1.3 Bagging avec différents modèles de base
print("\n🔄 Bagging avec SVM...")
bagging_svm = BaggingClassifier(
    estimator=SVC(kernel='rbf', probability=True, random_state=42),
    n_estimators=50,            # Moins d'estimateurs car SVM est coûteux
    max_samples=0.8,            # 80% des échantillons par modèle
    max_features=0.8,           # 80% des features par modèle
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

evaluate_model(
    bagging_svm, 
    X_train_features_scaled, 
    X_val_features_scaled, 
    y_train, 
    y_val, 
    'Bagging SVM'
)

# 5.1.4 Bagging avec Logistic Regression
print("\n🔄 Bagging avec Logistic Regression...")
bagging_lr = BaggingClassifier(
    estimator=LogisticRegression(max_iter=1000, random_state=42),
    n_estimators=100,
    max_samples=0.8,
    max_features=0.8,
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

evaluate_model(
    bagging_lr, 
    X_train_features_scaled, 
    X_val_features_scaled, 
    y_train, 
    y_val, 
    'Bagging Logistic Regression'
)

print("\n✅ Méthodes de Bagging entraînées!")

In [ ]:
# ===================================
# 5.2 ANALYSE DE L'IMPORTANCE DES FEATURES (BAGGING)
# ===================================

def analyze_feature_importance(model, feature_names, model_name, top_n=20):
    """Analyse l'importance des features pour les modèles tree-based"""
    
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        
        # Création du DataFrame
        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        # Visualisation
        plt.figure(figsize=(12, 8))
        top_features = feature_importance_df.head(top_n)
        
        sns.barplot(data=top_features, x='importance', y='feature', palette='viridis')
        plt.title(f'Top {top_n} Features - {model_name}', fontsize=14, fontweight='bold')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.show()
        
        return feature_importance_df
    else:
        print(f"❌ {model_name} ne supporte pas l'analyse d'importance des features")
        return None

# Génération des noms de features
feature_names = (
    ['mean', 'std', 'min', 'max', 'q25', 'q50', 'q75'] +  # Statistiques
    [f'hist_{i}' for i in range(16)] +                     # Histogramme
    ['grad_mean', 'grad_std'] +                            # Gradient
    ['cx', 'cy'] +                                         # Moments
    ['entropy', 'energy']                                  # Texture
)

# Analyse pour Random Forest optimisé
print("🔍 Analyse d'importance des features - Random Forest...")
rf_model = baseline_results['Random Forest Optimized']['model']
rf_importance = analyze_feature_importance(rf_model, feature_names, 'Random Forest Optimized')

# Analyse pour Extra Trees
print("🔍 Analyse d'importance des features - Extra Trees...")
et_model = baseline_results['Extra Trees']['model']
et_importance = analyze_feature_importance(et_model, feature_names, 'Extra Trees')

# Comparaison des importances
if rf_importance is not None and et_importance is not None:
    print("\n📊 Comparaison des top 10 features:")
    comparison_df = pd.merge(
        rf_importance.head(10)[['feature', 'importance']].rename(columns={'importance': 'RF_importance'}),
        et_importance.head(10)[['feature', 'importance']].rename(columns={'importance': 'ET_importance'}),
        on='feature',
        how='outer'
    ).fillna(0)
    
    print(comparison_df)

## 🚀 Section 6: Boosting Ensemble Methods

In [ ]:
# ===================================
# 6.1 IMPLÉMENTATION DES MÉTHODES DE BOOSTING
# ===================================

print("🚀 Implémentation des méthodes de Boosting...")

# 6.1.1 AdaBoost
print("\n🔄 AdaBoost...")
ada_boost = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=3, random_state=42),
    n_estimators=100,
    learning_rate=1.0,
    algorithm='SAMME',  # Supporte multiclass
    random_state=42
)

evaluate_model(
    ada_boost, 
    X_train_features_scaled, 
    X_val_features_scaled, 
    y_train, 
    y_val, 
    'AdaBoost'
)

# 6.1.2 Gradient Boosting
print("\n🔄 Gradient Boosting...")
gradient_boost = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=5,
    min_samples_leaf=2,
    subsample=0.8,              # Stochastic gradient boosting
    random_state=42
)

evaluate_model(
    gradient_boost, 
    X_train_features_scaled, 
    X_val_features_scaled, 
    y_train, 
    y_val, 
    'Gradient Boosting'
)

# 6.1.3 XGBoost (si disponible)
try:
    print("\n🔄 XGBoost...")
    xgb_classifier = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=3,
        min_child_weight=1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='mlogloss'  # Pour multiclass
    )
    
    evaluate_model(
        xgb_classifier, 
        X_train_features_scaled, 
        X_val_features_scaled, 
        y_train, 
        y_val, 
        'XGBoost'
    )
    
except NameError:
    print("⚠️ XGBoost non disponible - passage ignoré")

print("\n✅ Méthodes de Boosting entraînées!")

In [ ]:
# ===================================
# 6.2 COMPARAISON BAGGING VS BOOSTING
# ===================================

def create_ensemble_comparison():
    """Compare les performances des méthodes d'ensemble"""
    
    # Récupération des métriques pour comparaison
    comparison_data = []
    
    for model_name, results in baseline_results.items():
        comparison_data.append({
            'Model': model_name,
            'Type': get_model_type(model_name),
            'Val_Accuracy': results['val_accuracy'],
            'F1_Score': results['f1_score'],
            'Training_Time': results['training_time']
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    
    return df_comparison

def get_model_type(model_name):
    """Détermine le type de modèle"""
    if any(keyword in model_name.lower() for keyword in ['random forest', 'extra trees', 'bagging']):
        return 'Bagging'
    elif any(keyword in model_name.lower() for keyword in ['ada', 'gradient', 'xgboost']):
        return 'Boosting'
    else:
        return 'Baseline'

# Création de la comparaison
df_comparison = create_ensemble_comparison()

print("📊 Comparaison des méthodes d'ensemble:")
print(df_comparison.sort_values('Val_Accuracy', ascending=False))

# Visualisation comparative
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comparaison des Méthodes d\'Ensemble', fontsize=16, fontweight='bold')

# Accuracy par type
sns.boxplot(data=df_comparison, x='Type', y='Val_Accuracy', ax=axes[0, 0])
axes[0, 0].set_title('Accuracy par Type de Méthode')
axes[0, 0].set_ylabel('Validation Accuracy')

# F1-Score par type
sns.boxplot(data=df_comparison, x='Type', y='F1_Score', ax=axes[0, 1])
axes[0, 1].set_title('F1-Score par Type de Méthode')
axes[0, 1].set_ylabel('F1 Score')

# Temps d'entraînement par type
sns.boxplot(data=df_comparison, x='Type', y='Training_Time', ax=axes[1, 0])
axes[1, 0].set_title('Temps d\'Entraînement par Type')
axes[1, 0].set_ylabel('Training Time (s)')

# Correlation Accuracy vs Training Time
sns.scatterplot(data=df_comparison, x='Training_Time', y='Val_Accuracy', 
                hue='Type', size='F1_Score', sizes=(50, 200), ax=axes[1, 1])
axes[1, 1].set_title('Accuracy vs Temps d\'Entraînement')
axes[1, 1].set_xlabel('Training Time (s)')
axes[1, 1].set_ylabel('Validation Accuracy')

plt.tight_layout()
plt.show()

# Statistiques par type
print("\n📈 Statistiques par type de méthode:")
type_stats = df_comparison.groupby('Type').agg({
    'Val_Accuracy': ['mean', 'std', 'max'],
    'F1_Score': ['mean', 'std', 'max'],
    'Training_Time': ['mean', 'std', 'min']
}).round(4)

print(type_stats)

## 🧠 Section 7: Deep Learning Model Setup

In [ ]:
# ===================================
# 7.1 ARCHITECTURE CNN CUSTOM
# ===================================

def create_custom_cnn(input_shape=(224, 224, 3), num_classes=4):
    """
    Crée une architecture CNN personnalisée pour la classification d'images médicales
    """
    
    model = models.Sequential([
        # Bloc 1: Extraction de features de bas niveau
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloc 2: Features de niveau intermédiaire
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloc 3: Features de haut niveau
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloc 4: Features complexes
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Classification
        layers.GlobalAveragePooling2D(),  # Alternative à Flatten + Dense
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Création du modèle custom
print("🧠 Création du modèle CNN personnalisé...")
custom_cnn = create_custom_cnn(input_shape=(*IMG_SIZE, 3), num_classes=len(CLASSES))

# Affichage de l'architecture
print("📋 Architecture du modèle:")
custom_cnn.summary()

# Visualisation de l'architecture
tf.keras.utils.plot_model(
    custom_cnn, 
    to_file=str(RESULTS_DIR / "custom_cnn_architecture.png"),
    show_shapes=True, 
    show_layer_names=True,
    rankdir='TB'
)

print(f"✅ Diagramme d'architecture sauvegardé: {RESULTS_DIR / 'custom_cnn_architecture.png'}")

# Configuration de l'optimiseur et compilation
optimizer = optimizers.Adam(learning_rate=LEARNING_RATE)

custom_cnn.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

print("✅ Modèle compilé avec succès")

In [ ]:
# ===================================
# 7.2 CALLBACKS ET MONITORING
# ===================================

# Configuration des callbacks
def setup_callbacks(model_name, patience=10):
    """Configure les callbacks pour l'entraînement"""
    
    # Répertoire pour sauvegarder les modèles
    model_save_dir = MODELS_DIR / model_name
    model_save_dir.mkdir(exist_ok=True)
    
    callbacks_list = [
        # Early Stopping
        EarlyStopping(
            monitor='val_accuracy',
            patience=patience,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Réduction du learning rate
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-7,
            verbose=1
        ),
        
        # Sauvegarde du meilleur modèle
        ModelCheckpoint(
            filepath=str(model_save_dir / "best_model.h5"),
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        )
    ]
    
    return callbacks_list

# Configuration des callbacks pour le modèle custom
custom_callbacks = setup_callbacks("custom_cnn", patience=10)

print("✅ Callbacks configurés:")
for callback in custom_callbacks:
    print(f"   📋 {callback.__class__.__name__}")

# Préparation des générateurs de données
print("\n🔄 Préparation des générateurs de données...")

# Générateur d'entraînement avec augmentation
train_generator = train_datagen.flow(
    X_train_processed,
    y_train_categorical,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# Générateur de validation
val_generator = val_datagen.flow(
    X_val_processed,
    y_val_categorical,
    batch_size=BATCH_SIZE,
    shuffle=False
)

print(f"✅ Générateurs préparés:")
print(f"   📊 Train: {len(train_generator)} batches de {BATCH_SIZE}")
print(f"   📊 Validation: {len(val_generator)} batches de {BATCH_SIZE}")

# Fonction d'entraînement
def train_deep_model(model, model_name, train_gen, val_gen, epochs=EPOCHS, callbacks=None):
    """Entraîne un modèle de deep learning avec monitoring"""
    
    print(f"\n🚀 Entraînement de {model_name}...")
    print(f"   📊 Époques: {epochs}")
    print(f"   📊 Batch size: {BATCH_SIZE}")
    
    # Entraînement
    history = model.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=callbacks,
        verbose=1
    )
    
    print(f"✅ Entraînement de {model_name} terminé!")
    
    return history

print("🔧 Environnement d'entraînement Deep Learning configuré")

## 🔄 Section 8: Transfer Learning Implementation

In [ ]:
# ===================================
# 8.1 MODÈLES PRÉ-ENTRAÎNÉS AVEC TRANSFER LEARNING
# ===================================

def create_transfer_learning_model(base_model_name, input_shape=(224, 224, 3), num_classes=4, 
                                 freeze_base=True, trainable_layers=0):
    """
    Crée un modèle de transfer learning
    
    Args:
        base_model_name: 'VGG16', 'ResNet50', 'EfficientNetB0', ou 'InceptionV3'
        freeze_base: Si True, gèle les couches du modèle de base
        trainable_layers: Nombre de couches à rendre entraînables (depuis la fin)
    """
    
    # Sélection du modèle de base
    if base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'EfficientNetB0':
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'InceptionV3':
        base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError(f"Modèle {base_model_name} non supporté")
    
    # Gel des couches
    if freeze_base:
        base_model.trainable = False
    else:
        # Rendre seulement les dernières couches entraînables
        if trainable_layers > 0:
            base_model.trainable = True
            for layer in base_model.layers[:-trainable_layers]:
                layer.trainable = False
    
    # Construction du modèle complet
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Dictionnaire pour stocker les modèles de transfer learning
transfer_models = {}

print("🔄 Création des modèles de Transfer Learning...")

In [ ]:
# ===================================
# 8.2 ENTRAÎNEMENT DES MODÈLES DE TRANSFER LEARNING
# ===================================

# Configuration des modèles à tester
models_config = [
    {'name': 'VGG16_frozen', 'base': 'VGG16', 'freeze': True, 'trainable': 0},
    {'name': 'ResNet50_frozen', 'base': 'ResNet50', 'freeze': True, 'trainable': 0},
    {'name': 'EfficientNetB0_frozen', 'base': 'EfficientNetB0', 'freeze': True, 'trainable': 0}
]

# Stockage des historiques d'entraînement
training_histories = {}

for config in models_config:
    print(f"\n🔄 Configuration: {config['name']}")
    
    try:
        # Création du modèle
        model = create_transfer_learning_model(
            base_model_name=config['base'],
            input_shape=(*IMG_SIZE, 3),
            num_classes=len(CLASSES),
            freeze_base=config['freeze'],
            trainable_layers=config['trainable']
        )
        
        # Compilation
        model.compile(
            optimizer=optimizers.Adam(learning_rate=LEARNING_RATE),
            loss='categorical_crossentropy',
            metrics=['accuracy', 'precision', 'recall']
        )
        
        # Stockage
        transfer_models[config['name']] = model
        
        print(f"✅ {config['name']} créé et compilé")
        print(f"   📊 Paramètres entraînables: {model.count_params():,}")
        
        # Affichage du résumé pour le premier modèle
        if config['name'] == 'VGG16_frozen':
            print(f"\n📋 Exemple d'architecture - {config['name']}:")
            model.summary()
    
    except Exception as e:
        print(f"❌ Erreur lors de la création de {config['name']}: {e}")

print(f"\n✅ {len(transfer_models)} modèles de transfer learning créés")

# Entraînement d'un modèle de démonstration (VGG16)
if 'VGG16_frozen' in transfer_models:
    print("\n🚀 Entraînement de démonstration - VGG16 (epochs réduits)...")
    
    # Callbacks pour la démo
    demo_callbacks = setup_callbacks("VGG16_frozen_demo", patience=5)
    
    # Entraînement avec moins d'époques pour la démo
    demo_history = train_deep_model(
        transfer_models['VGG16_frozen'],
        "VGG16_frozen",
        train_generator,
        val_generator,
        epochs=10,  # Réduit pour la démo
        callbacks=demo_callbacks
    )
    
    training_histories['VGG16_frozen'] = demo_history
    
    print("✅ Entraînement de démonstration terminé")

print("\n💡 Note: Pour un entraînement complet, ajustez le nombre d'époques selon vos ressources")

## ⚡ Section 9: Fine-Tuning Pre-trained Models

In [ ]:
# ===================================
# 9.1 STRATÉGIES DE FINE-TUNING
# ===================================

def fine_tune_model(base_model_name, pretrained_model_path=None, 
                   unfreeze_layers=20, fine_tune_lr=1e-5):
    """
    Implémente le fine-tuning d'un modèle pré-entraîné
    
    Args:
        base_model_name: Nom du modèle de base
        pretrained_model_path: Chemin vers le modèle pré-entraîné (optionnel)
        unfreeze_layers: Nombre de couches à dégeler pour le fine-tuning
        fine_tune_lr: Learning rate réduit pour le fine-tuning
    """
    
    print(f"⚡ Fine-tuning de {base_model_name}")
    
    # Si un modèle pré-entraîné existe, le charger
    if pretrained_model_path and os.path.exists(pretrained_model_path):
        print(f"📂 Chargement du modèle pré-entraîné: {pretrained_model_path}")
        model = keras.models.load_model(pretrained_model_path)
    else:
        # Créer un nouveau modèle si pas de modèle pré-entraîné
        print("🔧 Création d'un nouveau modèle pour le fine-tuning")
        model = create_transfer_learning_model(
            base_model_name=base_model_name,
            input_shape=(*IMG_SIZE, 3),
            num_classes=len(CLASSES),
            freeze_base=False,
            trainable_layers=unfreeze_layers
        )
    
    # Phase 1: Dégel progressif des couches
    print(f"🔓 Dégel des {unfreeze_layers} dernières couches du modèle de base")
    
    # Identification du modèle de base (première couche)
    base_model = model.layers[0]
    
    # Gel de toutes les couches d'abord
    base_model.trainable = True
    
    # Gel des premières couches, dégel des dernières
    total_layers = len(base_model.layers)
    freeze_until = total_layers - unfreeze_layers
    
    for i, layer in enumerate(base_model.layers):
        if i < freeze_until:
            layer.trainable = False
        else:
            layer.trainable = True
    
    print(f"   📊 Couches totales: {total_layers}")
    print(f"   🔒 Couches gelées: {freeze_until}")
    print(f"   🔓 Couches entraînables: {unfreeze_layers}")
    
    # Recompilation avec learning rate réduit
    model.compile(
        optimizer=optimizers.Adam(learning_rate=fine_tune_lr),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    print(f"✅ Modèle recompilé avec learning rate: {fine_tune_lr}")
    
    return model

# Démonstration du fine-tuning sur VGG16
print("⚡ Démonstration de Fine-Tuning - VGG16")

# Stratégies de fine-tuning à tester
fine_tuning_strategies = [
    {
        'name': 'VGG16_fine_tuned_conservative',
        'base': 'VGG16',
        'unfreeze_layers': 10,
        'lr': 1e-5,
        'description': 'Fine-tuning conservateur - 10 dernières couches'
    },
    {
        'name': 'VGG16_fine_tuned_aggressive', 
        'base': 'VGG16',
        'unfreeze_layers': 20,
        'lr': 5e-6,
        'description': 'Fine-tuning agressif - 20 dernières couches'
    }
]

# Stockage des modèles fine-tunés
fine_tuned_models = {}

for strategy in fine_tuning_strategies:
    print(f"\n🔄 {strategy['description']}")
    
    try:
        # Création du modèle fine-tuné
        ft_model = fine_tune_model(
            base_model_name=strategy['base'],
            unfreeze_layers=strategy['unfreeze_layers'],
            fine_tune_lr=strategy['lr']
        )
        
        fine_tuned_models[strategy['name']] = ft_model
        
        # Affichage des informations sur l'entraînabilité
        trainable_count = sum([1 for layer in ft_model.layers for sublayer in layer.layers if hasattr(sublayer, 'trainable') and sublayer.trainable])
        total_count = sum([1 for layer in ft_model.layers for sublayer in layer.layers if hasattr(sublayer, 'trainable')])
        
        print(f"   📊 Couches entraînables: {trainable_count}/{total_count}")
        print(f"   📊 Paramètres entraînables: {ft_model.count_params():,}")
        
    except Exception as e:
        print(f"❌ Erreur lors du fine-tuning de {strategy['name']}: {e}")

print(f"\n✅ {len(fine_tuned_models)} modèles fine-tunés créés")

In [ ]:
# ===================================
# 9.2 LEARNING RATE SCHEDULING AVANCÉ
# ===================================

def create_advanced_callbacks(model_name, strategy_type="fine_tuning"):
    """
    Crée des callbacks avancés pour le fine-tuning
    """
    
    model_save_dir = MODELS_DIR / f"{model_name}_{strategy_type}"
    model_save_dir.mkdir(exist_ok=True)
    
    # Learning rate scheduler personnalisé
    def lr_schedule(epoch, lr):
        """Planification du learning rate"""
        if epoch < 5:
            return lr
        elif epoch < 15:
            return lr * 0.9
        else:
            return lr * 0.95
    
    callbacks_list = [
        # Early Stopping plus patient pour le fine-tuning
        EarlyStopping(
            monitor='val_accuracy',
            patience=15,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Réduction automatique du learning rate
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,
            patience=7,
            min_lr=1e-8,
            verbose=1
        ),
        
        # Learning rate scheduler personnalisé
        keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1),
        
        # Sauvegarde des meilleurs modèles
        ModelCheckpoint(
            filepath=str(model_save_dir / "best_model.h5"),
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        ),
        
        # Sauvegarde des checkpoints
        ModelCheckpoint(
            filepath=str(model_save_dir / "checkpoint_epoch_{epoch:02d}.h5"),
            save_freq='epoch',
            save_weights_only=True,
            verbose=0
        )
    ]
    
    return callbacks_list

# Démonstration d'entraînement avec fine-tuning (version réduite)
if 'VGG16_fine_tuned_conservative' in fine_tuned_models:
    print("🚀 Démonstration de Fine-Tuning avec callbacks avancés...")
    
    # Configuration des callbacks avancés
    ft_callbacks = create_advanced_callbacks("VGG16", "fine_tuning_demo")
    
    # Entraînement avec paramètres ajustés pour le fine-tuning
    print("📊 Configuration pour Fine-Tuning:")
    print("   - Learning rate réduit")
    print("   - Patience augmentée")
    print("   - Callbacks avancés")
    print("   - Epochs réduits pour démo")
    
    # Note: Dans un scénario réel, vous entraîneriez ici le modèle
    print("\n💡 Note: Dans un entraînement complet, vous exécuteriez:")
    print("   ft_history = train_deep_model(model, 'VGG16_fine_tuned', train_gen, val_gen, epochs=30)")
    
    print("✅ Configuration de fine-tuning préparée")

print("\n🔧 Pipeline de Fine-Tuning configuré avec succès!")

## 🤝 Section 10: Ensemble of Deep Learning Models

In [ ]:
# ===================================
# 10.1 ENSEMBLE VOTING POUR DEEP LEARNING
# ===================================

class DeepLearningEnsemble:
    """
    Classe pour créer des ensembles de modèles de deep learning
    """
    
    def __init__(self, models_dict, ensemble_method='soft_voting'):
        """
        Args:
            models_dict: Dictionnaire {nom: modèle} des modèles à ensembler
            ensemble_method: 'soft_voting', 'hard_voting', ou 'stacking'
        """
        self.models = models_dict
        self.ensemble_method = ensemble_method
        self.model_names = list(models_dict.keys())
        
    def predict_ensemble(self, X, return_individual=False):
        """
        Fait des prédictions avec l'ensemble
        """
        individual_predictions = {}
        all_predictions = []
        
        print(f"🔄 Prédiction avec {len(self.models)} modèles...")
        
        # Prédictions individuelles
        for name, model in self.models.items():
            try:
                pred = model.predict(X, verbose=0)
                individual_predictions[name] = pred
                all_predictions.append(pred)
                print(f"   ✅ {name}: {pred.shape}")
            except Exception as e:
                print(f"   ❌ Erreur avec {name}: {e}")
                continue
        
        if not all_predictions:
            raise ValueError("Aucune prédiction réussie")
        
        # Ensemble des prédictions
        all_predictions = np.array(all_predictions)
        
        if self.ensemble_method == 'soft_voting':
            # Moyenne des probabilités
            ensemble_pred = np.mean(all_predictions, axis=0)
        elif self.ensemble_method == 'hard_voting':
            # Vote majoritaire
            individual_classes = [np.argmax(pred, axis=1) for pred in all_predictions]
            ensemble_classes = []
            for i in range(len(X)):
                votes = [pred[i] for pred in individual_classes]
                ensemble_classes.append(max(set(votes), key=votes.count))
            
            # Conversion en format one-hot
            ensemble_pred = to_categorical(ensemble_classes, num_classes=len(CLASSES))
        
        else:
            raise ValueError(f"Méthode d'ensemble {self.ensemble_method} non supportée")
        
        if return_individual:
            return ensemble_pred, individual_predictions
        return ensemble_pred
    
    def evaluate_ensemble(self, X, y_true, return_individual=False):
        """
        Évalue les performances de l'ensemble
        """
        print(f"📊 Évaluation de l'ensemble ({self.ensemble_method})...")
        
        # Prédictions
        if return_individual:
            ensemble_pred, individual_preds = self.predict_ensemble(X, return_individual=True)
        else:
            ensemble_pred = self.predict_ensemble(X)
            individual_preds = None
        
        # Conversion en classes pour les métriques
        y_true_classes = np.argmax(y_true, axis=1)
        ensemble_classes = np.argmax(ensemble_pred, axis=1)
        
        # Métriques de l'ensemble
        accuracy = accuracy_score(y_true_classes, ensemble_classes)
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true_classes, ensemble_classes, average='weighted'
        )
        
        results = {
            'ensemble_accuracy': accuracy,
            'ensemble_precision': precision,
            'ensemble_recall': recall,
            'ensemble_f1': f1,
            'ensemble_predictions': ensemble_pred
        }
        
        # Évaluation individuelle si demandée
        if return_individual and individual_preds:
            individual_results = {}
            for name, pred in individual_preds.items():
                pred_classes = np.argmax(pred, axis=1)
                ind_acc = accuracy_score(y_true_classes, pred_classes)
                individual_results[name] = {
                    'accuracy': ind_acc,
                    'predictions': pred
                }
            results['individual_results'] = individual_results
        
        return results

# Démonstration d'ensemble avec les modèles disponibles
print("🤝 Création d'ensembles de modèles Deep Learning...")

# Vérification des modèles disponibles pour l'ensemble
available_models = {}

# Ajout des modèles de transfer learning si disponibles
for model_name, model in transfer_models.items():
    available_models[model_name] = model

# Ajout des modèles fine-tunés si disponibles  
for model_name, model in fine_tuned_models.items():
    available_models[model_name] = model

print(f"📊 Modèles disponibles pour l'ensemble: {list(available_models.keys())}")

if len(available_models) >= 2:
    # Création d'ensembles avec différentes méthodes
    ensemble_configs = [
        {'method': 'soft_voting', 'description': 'Moyenne des probabilités'},
        {'method': 'hard_voting', 'description': 'Vote majoritaire'}
    ]
    
    ensembles = {}
    
    for config in ensemble_configs:
        ensemble_name = f"DL_Ensemble_{config['method']}"
        ensemble = DeepLearningEnsemble(
            models_dict=available_models,
            ensemble_method=config['method']
        )
        ensembles[ensemble_name] = ensemble
        
        print(f"✅ {ensemble_name} créé: {config['description']}")
    
    print(f"\n🎯 {len(ensembles)} ensembles de Deep Learning configurés")
    
else:
    print("⚠️ Pas assez de modèles entraînés pour créer un ensemble")
    print("   Entraînez d'abord plusieurs modèles de transfer learning")

print("\n💡 Note: L'évaluation complète des ensembles nécessite des modèles entièrement entraînés")

## ⚙️ Section 11: Hyperparameter Optimization

In [ ]:
# ===================================
# 11.1 OPTIMISATION DES HYPERPARAMÈTRES - MÉTHODES TRADITIONNELLES
# ===================================

print("⚙️ Optimisation des hyperparamètres pour les méthodes d'ensemble...")

# Configuration des espaces de recherche
param_grids = {
    'RandomForest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 15, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', None]
    },
    
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    } if 'xgb' in globals() else {},
    
    'GradientBoosting': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'subsample': [0.8, 0.9, 1.0]
    }
}

def optimize_hyperparameters(model_class, param_grid, X_train, y_train, 
                           cv=3, scoring='accuracy', n_jobs=-1, verbose=True):
    """
    Optimise les hyperparamètres avec GridSearchCV
    """
    if not param_grid:
        print("⚠️ Grille de paramètres vide - optimisation ignorée")
        return None, None
    
    print(f"🔍 Optimisation avec {len(param_grid)} paramètres...")
    print(f"   📊 Cross-validation: {cv} folds")
    print(f"   📊 Métrique: {scoring}")
    
    # Configuration de la recherche
    grid_search = GridSearchCV(
        estimator=model_class,
        param_grid=param_grid,
        cv=cv,
        scoring=scoring,
        n_jobs=n_jobs,
        return_train_score=True,
        verbose=1 if verbose else 0
    )
    
    # Exécution de la recherche
    start_time = datetime.now()
    grid_search.fit(X_train, y_train)
    optimization_time = datetime.now() - start_time
    
    if verbose:
        print(f"✅ Optimisation terminée en {optimization_time.total_seconds():.2f}s")
        print(f"🎯 Meilleur score: {grid_search.best_score_:.4f}")
        print(f"⚙️ Meilleurs paramètres: {grid_search.best_params_}")
    
    return grid_search.best_estimator_, grid_search.cv_results_

# Stockage des résultats d'optimisation
optimization_results = {}

# Optimisation pour Random Forest
print("\n🌳 Optimisation Random Forest...")
if 'RandomForest' in param_grids:
    # Grille réduite pour la démo
    rf_demo_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 15],
        'min_samples_split': [2, 5],
        'max_features': ['sqrt', None]
    }
    
    best_rf, rf_cv_results = optimize_hyperparameters(
        RandomForestClassifier(random_state=42, n_jobs=-1),
        rf_demo_grid,
        X_train_features_scaled,
        y_train,
        cv=3,
        verbose=True
    )
    
    if best_rf:
        optimization_results['RandomForest'] = {
            'best_model': best_rf,
            'cv_results': rf_cv_results
        }

# Optimisation pour Gradient Boosting
print("\n🚀 Optimisation Gradient Boosting...")
gb_demo_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.1, 0.2],
    'max_depth': [3, 5]
}

best_gb, gb_cv_results = optimize_hyperparameters(
    GradientBoostingClassifier(random_state=42),
    gb_demo_grid,
    X_train_features_scaled,
    y_train,
    cv=3,
    verbose=True
)

if best_gb:
    optimization_results['GradientBoosting'] = {
        'best_model': best_gb,
        'cv_results': gb_cv_results
    }

print(f"\n✅ {len(optimization_results)} modèles optimisés")

# Évaluation des modèles optimisés
print("\n📊 Évaluation des modèles optimisés...")
for model_name, results in optimization_results.items():
    model = results['best_model']
    
    # Évaluation sur validation
    val_pred = model.predict(X_val_features_scaled)
    val_accuracy = accuracy_score(y_val, val_pred)
    
    print(f"🎯 {model_name} optimisé:")
    print(f"   📊 Validation Accuracy: {val_accuracy:.4f}")
    
    # Comparaison avec le modèle baseline
    if model_name in baseline_results:
        baseline_acc = baseline_results[model_name]['val_accuracy']
        improvement = val_accuracy - baseline_acc
        print(f"   📈 Amélioration: {improvement:+.4f}")
    
    # Stockage dans baseline_results pour comparaison
    baseline_results[f"{model_name}_optimized"] = {
        'model': model,
        'val_accuracy': val_accuracy,
        'y_val_pred': val_pred
    }

In [ ]:
# ===================================
# 11.2 RECHERCHE ALÉATOIRE ET ANALYSE DES HYPERPARAMÈTRES
# ===================================

def randomized_hyperparameter_search(model_class, param_distributions, 
                                    X_train, y_train, n_iter=20, cv=3):
    """
    Recherche aléatoire d'hyperparamètres avec RandomizedSearchCV
    """
    print(f"🎲 Recherche aléatoire avec {n_iter} itérations...")
    
    random_search = RandomizedSearchCV(
        estimator=model_class,
        param_distributions=param_distributions,
        n_iter=n_iter,
        cv=cv,
        scoring='accuracy',
        n_jobs=-1,
        random_state=42,
        return_train_score=True,
        verbose=1
    )
    
    start_time = datetime.now()
    random_search.fit(X_train, y_train)
    search_time = datetime.now() - start_time
    
    print(f"✅ Recherche terminée en {search_time.total_seconds():.2f}s")
    print(f"🎯 Meilleur score: {random_search.best_score_:.4f}")
    print(f"⚙️ Meilleurs paramètres: {random_search.best_params_}")
    
    return random_search.best_estimator_, random_search.cv_results_

# Distributions pour la recherche aléatoire
from scipy.stats import randint, uniform

rf_param_distributions = {
    'n_estimators': randint(50, 300),
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

# Recherche aléatoire pour Random Forest
print("\n🎲 Recherche aléatoire - Random Forest...")
best_rf_random, rf_random_results = randomized_hyperparameter_search(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    rf_param_distributions,
    X_train_features_scaled,
    y_train,
    n_iter=10,  # Réduit pour la démo
    cv=3
)

# Analyse des résultats de recherche
def analyze_hyperparameter_results(cv_results, param_name, model_name):
    """
    Analyse l'impact d'un hyperparamètre sur les performances
    """
    if f'param_{param_name}' not in cv_results:
        print(f"⚠️ Paramètre {param_name} non trouvé dans les résultats")
        return
    
    # Extraction des données
    param_values = cv_results[f'param_{param_name}']
    mean_scores = cv_results['mean_test_score']
    
    # Création du DataFrame pour l'analyse
    df_analysis = pd.DataFrame({
        'param_value': param_values,
        'mean_score': mean_scores
    })
    
    # Gestion des valeurs None
    df_analysis = df_analysis[df_analysis['param_value'].notna()]
    
    if len(df_analysis) == 0:
        print(f"⚠️ Pas de données valides pour {param_name}")
        return
    
    # Visualisation
    plt.figure(figsize=(10, 6))
    
    if df_analysis['param_value'].dtype == 'object':
        # Paramètre catégoriel
        df_grouped = df_analysis.groupby('param_value')['mean_score'].agg(['mean', 'std']).reset_index()
        plt.bar(range(len(df_grouped)), df_grouped['mean'], 
                yerr=df_grouped['std'], alpha=0.7, capsize=5)
        plt.xticks(range(len(df_grouped)), df_grouped['param_value'], rotation=45)
    else:
        # Paramètre numérique
        plt.scatter(df_analysis['param_value'], df_analysis['mean_score'], alpha=0.6)
        plt.xlabel(param_name)
    
    plt.title(f'Impact de {param_name} sur les performances - {model_name}')
    plt.ylabel('Score de validation')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Analyse des résultats de Random Forest
if rf_random_results:
    print("\n📊 Analyse des hyperparamètres - Random Forest...")
    
    # Analyse de différents paramètres
    important_params = ['n_estimators', 'max_depth', 'min_samples_split']
    
    for param in important_params:
        analyze_hyperparameter_results(rf_random_results, param, 'Random Forest')

# Comparaison des méthodes d'optimisation
print("\n📈 Comparaison des méthodes d'optimisation:")

comparison_data = []

# Baseline
for model_name in ['Random Forest', 'Gradient Boosting']:
    if model_name in baseline_results:
        comparison_data.append({
            'Model': model_name,
            'Method': 'Baseline',
            'Accuracy': baseline_results[model_name]['val_accuracy']
        })

# Optimisés
for model_name in ['RandomForest', 'GradientBoosting']:
    if f"{model_name}_optimized" in baseline_results:
        comparison_data.append({
            'Model': model_name,
            'Method': 'Grid Search',
            'Accuracy': baseline_results[f"{model_name}_optimized"]['val_accuracy']
        })

# Random Search (Random Forest seulement)
if best_rf_random:
    rf_random_acc = accuracy_score(y_val, best_rf_random.predict(X_val_features_scaled))
    comparison_data.append({
        'Model': 'RandomForest',
        'Method': 'Random Search',
        'Accuracy': rf_random_acc
    })

if comparison_data:
    df_optimization_comparison = pd.DataFrame(comparison_data)
    
    # Visualisation
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_optimization_comparison, x='Model', y='Accuracy', hue='Method')
    plt.title('Comparaison des Méthodes d\'Optimisation')
    plt.ylabel('Validation Accuracy')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    print("📊 Résultats de comparaison:")
    print(df_optimization_comparison)

print("\n✅ Optimisation des hyperparamètres terminée")

## 📊 Section 12: Model Evaluation and Comparison

In [ ]:
# ===================================
# 12.1 MÉTRIQUES AVANCÉES ET MATRICES DE CONFUSION
# ===================================

def comprehensive_model_evaluation(y_true, y_pred, model_name, class_names):
    """
    Évaluation complète d'un modèle avec métriques avancées
    """
    print(f"\n📊 Évaluation complète - {model_name}")
    print("=" * 50)
    
    # Métriques globales
    accuracy = accuracy_score(y_true, y_pred)
    precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    
    print(f"🎯 Accuracy: {accuracy:.4f}")
    print(f"🎯 Precision (weighted): {precision:.4f}")
    print(f"🎯 Recall (weighted): {recall:.4f}")
    print(f"🎯 F1-Score (weighted): {f1:.4f}")
    
    # Métriques par classe
    print(f"\n📋 Rapport de classification:")
    print(classification_report(y_true, y_pred, target_names=class_names))
    
    # Matrice de confusion
    cm = confusion_matrix(y_true, y_pred)
    
    # Visualisation de la matrice de confusion
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Matrice de Confusion - {model_name}')
    plt.xlabel('Prédictions')
    plt.ylabel('Vraies étiquettes')
    plt.tight_layout()
    plt.show()
    
    # Métriques par classe détaillées
    class_metrics = {}
    for i, class_name in enumerate(class_names):
        class_precision = precision_recall_fscore_support(y_true, y_pred, labels=[i], average=None)[0][0]
        class_recall = precision_recall_fscore_support(y_true, y_pred, labels=[i], average=None)[1][0]
        class_f1 = precision_recall_fscore_support(y_true, y_pred, labels=[i], average=None)[2][0]
        
        class_metrics[class_name] = {
            'precision': class_precision,
            'recall': class_recall,
            'f1_score': class_f1,
            'support': support[i]
        }
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm,
        'class_metrics': class_metrics
    }

# Évaluation de tous les modèles disponibles
print("📊 Évaluation complète de tous les modèles...")

evaluation_results = {}

# Évaluation des modèles baseline
for model_name, results in baseline_results.items():
    if 'y_val_pred' in results:
        eval_results = comprehensive_model_evaluation(
            y_val, 
            results['y_val_pred'], 
            model_name, 
            CLASSES
        )
        evaluation_results[model_name] = eval_results

In [ ]:
# ===================================
# 12.2 COMPARAISON GLOBALE DES MODÈLES
# ===================================

def create_models_comparison_dashboard():
    """
    Crée un dashboard de comparaison des modèles
    """
    if not evaluation_results:
        print("⚠️ Aucun résultat d'évaluation disponible")
        return
    
    # Compilation des métriques
    comparison_data = []
    for model_name, results in evaluation_results.items():
        comparison_data.append({
            'Model': model_name,
            'Accuracy': results['accuracy'],
            'Precision': results['precision'],
            'Recall': results['recall'],
            'F1_Score': results['f1_score'],
            'Type': get_model_type(model_name)
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    
    # Tri par accuracy
    df_comparison = df_comparison.sort_values('Accuracy', ascending=False)
    
    print("🏆 Classement des modèles par Accuracy:")
    print(df_comparison[['Model', 'Accuracy', 'F1_Score', 'Type']].to_string(index=False))
    
    # Visualisations
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Dashboard de Comparaison des Modèles', fontsize=16, fontweight='bold')
    
    # 1. Accuracy par modèle
    df_sorted = df_comparison.sort_values('Accuracy', ascending=True)
    bars = axes[0, 0].barh(range(len(df_sorted)), df_sorted['Accuracy'], 
                          color=plt.cm.viridis(np.linspace(0, 1, len(df_sorted))))
    axes[0, 0].set_yticks(range(len(df_sorted)))
    axes[0, 0].set_yticklabels(df_sorted['Model'], fontsize=8)
    axes[0, 0].set_xlabel('Accuracy')
    axes[0, 0].set_title('Accuracy par Modèle')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Ajout des valeurs sur les barres
    for i, bar in enumerate(bars):
        width = bar.get_width()
        axes[0, 0].text(width + 0.001, bar.get_y() + bar.get_height()/2, 
                       f'{width:.3f}', ha='left', va='center', fontsize=8)
    
    # 2. F1-Score vs Accuracy
    scatter = axes[0, 1].scatter(df_comparison['Accuracy'], df_comparison['F1_Score'], 
                                c=df_comparison.index, cmap='viridis', s=100, alpha=0.7)
    axes[0, 1].set_xlabel('Accuracy')
    axes[0, 1].set_ylabel('F1-Score')
    axes[0, 1].set_title('F1-Score vs Accuracy')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Ligne diagonale de référence
    min_val = min(df_comparison['Accuracy'].min(), df_comparison['F1_Score'].min())
    max_val = max(df_comparison['Accuracy'].max(), df_comparison['F1_Score'].max())
    axes[0, 1].plot([min_val, max_val], [min_val, max_val], 'k--', alpha=0.5)
    
    # 3. Métriques par type de modèle
    df_melted = df_comparison.melt(id_vars=['Model', 'Type'], 
                                  value_vars=['Accuracy', 'Precision', 'Recall', 'F1_Score'],
                                  var_name='Metric', value_name='Score')
    
    sns.boxplot(data=df_melted, x='Type', y='Score', hue='Metric', ax=axes[1, 0])
    axes[1, 0].set_title('Distribution des Métriques par Type')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # 4. Heatmap des corrélations entre métriques
    metrics_corr = df_comparison[['Accuracy', 'Precision', 'Recall', 'F1_Score']].corr()
    sns.heatmap(metrics_corr, annot=True, cmap='coolwarm', center=0, 
                square=True, ax=axes[1, 1])
    axes[1, 1].set_title('Corrélations entre Métriques')
    
    plt.tight_layout()
    plt.show()
    
    return df_comparison

# Création du dashboard
print("📊 Création du dashboard de comparaison...")
comparison_df = create_models_comparison_dashboard()

# Analyse des meilleures performances par catégorie
if comparison_df is not None and not comparison_df.empty:
    print("\n🏆 Meilleurs modèles par catégorie:")
    
    categories = ['Baseline', 'Bagging', 'Boosting']
    for category in categories:
        cat_models = comparison_df[comparison_df['Type'] == category]
        if not cat_models.empty:
            best_model = cat_models.loc[cat_models['Accuracy'].idxmax()]
            print(f"{category}: {best_model['Model']} (Accuracy: {best_model['Accuracy']:.4f})")
    
    # Modèle global champion
    best_overall = comparison_df.loc[comparison_df['Accuracy'].idxmax()]
    print(f"\n👑 Champion global: {best_overall['Model']}")
    print(f"   📊 Accuracy: {best_overall['Accuracy']:.4f}")
    print(f"   📊 F1-Score: {best_overall['F1_Score']:.4f}")
    print(f"   📊 Type: {best_overall['Type']}")

print("\n✅ Évaluation comparative terminée")

## 🎨 Section 13: Prediction Visualization

In [ ]:
# ===================================
# 13.1 VISUALISATION DES PRÉDICTIONS ET INTERPRÉTABILITÉ
# ===================================

def visualize_predictions_with_confidence(model, X_samples, y_true, sample_indices, 
                                        model_name, class_names, is_deep_learning=False):
    """
    Visualise les prédictions avec scores de confiance
    """
    n_samples = len(sample_indices)
    fig, axes = plt.subplots(2, n_samples//2, figsize=(16, 8))
    fig.suptitle(f'Prédictions et Confiance - {model_name}', fontsize=16, fontweight='bold')
    
    if is_deep_learning:
        # Pour les modèles de deep learning
        predictions_proba = model.predict(X_samples, verbose=0)
        predictions = np.argmax(predictions_proba, axis=1)
    else:
        # Pour les modèles traditionnels avec features
        X_features = extract_traditional_features(X_samples)
        X_features_scaled = scaler.transform(X_features)
        predictions_proba = model.predict_proba(X_features_scaled)
        predictions = model.predict(X_features_scaled)
    
    for idx, sample_idx in enumerate(sample_indices):
        row = idx // (n_samples//2)
        col = idx % (n_samples//2)
        
        # Image
        axes[row, col].imshow(X_samples[sample_idx])
        
        # Informations sur la prédiction
        true_label = class_names[y_true[sample_idx]]
        pred_label = class_names[predictions[sample_idx]]
        confidence = np.max(predictions_proba[sample_idx])
        
        # Couleur selon la justesse de la prédiction
        color = 'green' if true_label == pred_label else 'red'
        
        title = f'Vraie: {true_label}\nPréd: {pred_label}\nConf: {confidence:.3f}'
        axes[row, col].set_title(title, color=color, fontsize=10)
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Graphique des scores de confiance par classe
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    for i, sample_idx in enumerate(sample_indices):
        proba_scores = predictions_proba[sample_idx]
        x_pos = np.arange(len(class_names)) + i * 0.1
        
        bars = ax.bar(x_pos, proba_scores, width=0.1, alpha=0.7, 
                     label=f'Échantillon {sample_idx+1}')
        
        # Highlight de la classe prédite
        max_idx = np.argmax(proba_scores)
        bars[max_idx].set_color('red')
        bars[max_idx].set_alpha(1.0)
    
    ax.set_xlabel('Classes')
    ax.set_ylabel('Score de Confiance')
    ax.set_title(f'Scores de Confiance par Classe - {model_name}')
    ax.set_xticks(np.arange(len(class_names)) + 0.2)
    ax.set_xticklabels(class_names, rotation=45)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Démonstration avec le meilleur modèle traditionnel
if baseline_results and X_val_processed is not None:
    print("🎨 Visualisation des prédictions...")
    
    # Sélection d'échantillons pour la visualisation
    n_viz_samples = 6
    sample_indices = np.random.choice(len(X_val_processed), n_viz_samples, replace=False)
    
    # Récupération du meilleur modèle traditionnel
    best_traditional_model = None
    best_traditional_name = ""
    best_score = 0
    
    for model_name, results in baseline_results.items():
        if 'val_accuracy' in results and results['val_accuracy'] > best_score:
            best_score = results['val_accuracy']
            best_traditional_model = results['model']
            best_traditional_name = model_name
    
    if best_traditional_model:
        print(f"📊 Visualisation avec le meilleur modèle: {best_traditional_name}")
        
        # Échantillons pour la visualisation
        X_viz = X_val_processed[sample_indices]
        y_viz = y_val[sample_indices]
        
        # Visualisation
        visualize_predictions_with_confidence(
            best_traditional_model,
            X_viz,
            y_viz,
            range(len(sample_indices)),
            best_traditional_name,
            CLASSES,
            is_deep_learning=False
        )
    
    else:
        print("⚠️ Aucun modèle traditionnel disponible pour la visualisation")

print("✅ Visualisations créées")

## 🎯 Conclusion et Récapitulatif

### 📊 **Résumé du Notebook**

Ce notebook complet a démontré les techniques avancées de machine learning pour la classification d'images médicales COVID-19 :

#### 🔧 **Techniques Implémentées :**

1. **📦 Méthodes d'Ensemble - Bagging :**
   - Random Forest optimisé
   - Extra Trees (Extremely Randomized Trees)
   - Bagging avec SVM et Logistic Regression
   - Analyse d'importance des features

2. **🚀 Méthodes d'Ensemble - Boosting :**
   - AdaBoost avec arbres de décision
   - Gradient Boosting Machine (GBM)
   - XGBoost (si disponible)
   - Comparaison des performances

3. **🧠 Deep Learning :**
   - CNN personnalisé from scratch
   - Transfer Learning (VGG16, ResNet50, EfficientNetB0)
   - Fine-Tuning avec stratégies avancées
   - Ensemble de modèles de deep learning

4. **⚙️ Optimisation :**
   - Grid Search pour hyperparamètres
   - Random Search alternatif
   - Cross-validation stratifiée
   - Learning rate scheduling

5. **📊 Évaluation :**
   - Métriques complètes (Accuracy, Precision, Recall, F1)
   - Matrices de confusion détaillées
   - Comparaisons visuelles
   - Dashboard de performances

#### 💡 **Points Clés Appris :**

- **Bagging** : Réduit la variance, bon pour overfitting
- **Boosting** : Réduit le biais, séquentiel et puissant
- **Transfer Learning** : Efficace pour datasets médicaux limités
- **Fine-Tuning** : Améliore les performances avec attention aux learning rates
- **Ensemble Deep Learning** : Combine les forces de différents modèles

#### 🛠️ **Recommandations pour la Production :**

1. **Entraînement complet** : Utilisez plus d'époques (50-100) pour les modèles DL
2. **Validation externe** : Testez sur un dataset externe pour validation
3. **Monitoring** : Implémentez un suivi des performances en temps réel
4. **Explicabilité** : Ajoutez des techniques comme LIME ou GRAD-CAM
5. **Déploiement** : Considérez l'optimisation des modèles (quantization, pruning)

### 🎯 **Prochaines Étapes Suggérées :**

1. **Augmentation de données avancée** : Techniques spécifiques au médical
2. **Architecture personnalisée** : CNN adapté aux radiographies
3. **Méta-learning** : Apprentissage sur différents types d'images médicales
4. **Federated Learning** : Entraînement distribué préservant la confidentialité
5. **Validation clinique** : Collaboration avec professionnels de santé

---

**🏆 Félicitations ! Vous avez maintenant une base solide pour les techniques avancées de ML dans le domaine médical.**